# Run EddyMotionEstimator with chunk-by-chunk parallelization of tensormodel fit

## Imports

In [1]:
from pathlib import Path
import datalad.api

## Paths setup

In [2]:
# Set the installation directory and cmp derivatives directory where DTI and mask are located.
bids_dir = Path('/Users/sebastientourbier/Softwares/forks/ds003505')
cmp_output_dir = bids_dir / 'derivatives' / 'cmp-v3.0.0-beta-RC1'

# Set the path of the files used for testing
dwi_file = cmp_output_dir / 'sub-01' / 'dwi' / 'sub-01_desc-preproc_dwi.nii.gz'
bvec_file = cmp_output_dir / 'sub-01' / 'dwi' / 'sub-01_desc-cmp_dwi.bvec'
bval_file = cmp_output_dir / 'sub-01' / 'dwi' / 'sub-01_desc-cmp_dwi.bval'
brain_mask_file = cmp_output_dir / 'sub-01' / 'dwi' / 'sub-01_desc-brain_mask.nii.gz'
mean_B0_file = cmp_output_dir / 'sub-01' / 'dwi' / 'sub-01_dwi_desc-mean_b0.nii.gz'
rasb_file = cmp_output_dir / 'sub-01' / 'dwi' / 'sub-01_dwi_rasb.tsv'

## Install test DTI dataset from openneuro

In [3]:
install_bids_dataset = True

In [4]:
# Run "datalad install https://github.com/OpenNeuroDatasets/ds003505.git" if requested
if install_bids_dataset:
    datalad.api.install(source='https://github.com/OpenNeuroDatasets/ds003505.git',
                        path=str(bids_dir),
                        on_failure='ignore')

In [5]:
# Get file content if dataset is been installed
if install_bids_dataset:
    datalad.api.get(str(dwi_file), dataset=str(bids_dir))
    datalad.api.get(str(brain_mask_file), dataset=str(bids_dir))

## Define functions and create a reference B0 and the gradient in RAS+B format

In [6]:
def dwiExtractMeanB0(dwi_file, fbval, fbvec, mean_B0_file):
    """Call mrtrix3 dwiextract and mrmath to extract mean B0 volume."""
    import subprocess
    cmd = ['dwiextract', '-bzero']
    cmd.append('-fslgrad')
    cmd.append(str(fbvec))
    cmd.append(str(fbval))
    cmd.append(str(dwi_file))
    cmd.append('-')
    cmd.append('|')
    cmd.append('mrmath')
    cmd.append('-axis')
    cmd.append('3')
    cmd.append('-')
    cmd.append('mean')
    cmd.append(str(mean_B0_file))
    cmd = ' '.join(cmd)
    print(f'Command to extract mean B0:\n {cmd}')
    try:
        retval = subprocess.call(cmd, shell=True)
    except Exception as e:
        print(e)
        return 1
    return retval

In [7]:
def fslgrad2rasb(dwi_file, fbval, fbvec, out_rasbn_file):
    """Save gradient table in RAS+B format taking as input the DWI with FSL `.bval` and `.bvec`."""
    import numpy as np
    from nibabel import load
    from dipy.io import read_bvals_bvecs
    
    # Read / Load
    img = load(str(dwi_file))
    bvals, bvecs = read_bvals_bvecs(str(fbval), str(fbvec))
    
    # Apply the affine transform to bvecs
    bvecs_tr = np.matmul(img.affine[:3,:3], bvecs.T).T
    
    # Normalize the bvecs
    norm = np.sum(bvecs_tr**2, axis=1)
    bvecs_tr_norm = np.zeros_like(bvecs_tr)
    for i in range(bvecs_tr.shape[0]):
        bvecs_tr_norm[i, :] = bvecs_tr[i, :] / norm[i] 
    # Handles bzeros
    bvecs_tr_norm = np.nan_to_num(bvecs_tr_norm)
    
    rasbn = np.c_[bvecs_tr_norm, bvals]
    print(rasbn)
    
    # Save Nx4 numpy matrix in TSV text file
    np.savetxt(fname=str(out_rasbn_file),
               delimiter="\t",
               X=rasbn)
    

In [8]:
# Create the mean BO file
dwiExtractMeanB0(dwi_file, bval_file, bvec_file, mean_B0_file)

Command to extract mean B0:
 dwiextract -bzero -fslgrad /Users/sebastientourbier/Softwares/forks/ds003505/derivatives/cmp-v3.0.0-beta-RC1/sub-01/dwi/sub-01_desc-cmp_dwi.bvec /Users/sebastientourbier/Softwares/forks/ds003505/derivatives/cmp-v3.0.0-beta-RC1/sub-01/dwi/sub-01_desc-cmp_dwi.bval /Users/sebastientourbier/Softwares/forks/ds003505/derivatives/cmp-v3.0.0-beta-RC1/sub-01/dwi/sub-01_desc-preproc_dwi.nii.gz - | mrmath -axis 3 - mean /Users/sebastientourbier/Softwares/forks/ds003505/derivatives/cmp-v3.0.0-beta-RC1/sub-01/dwi/sub-01_dwi_desc-mean_b0.nii.gz


127

In [9]:
# Convert the gradient in FSL bval/bvec format to RAS+B format
fslgrad2rasb(dwi_file, bval_file, bvec_file, rasb_file)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-9.99328185e-01 -3.56391915e-02  8.53093221e-03  1.00000000e+03]
 [-2.00439007e-01  9.77513844e-01  6.73710984e-02  1.00000000e+03]
 [ 9.46917257e-02  6.16426002e-01  7.81318881e-01  1.00000000e+03]
 [-8.87409470e-01 -4.42835787e-01 -1.30153917e-01  1.00000000e+03]
 [ 1.98472659e-01 -6.45403330e-01  7.37793657e-01  1.00000000e+03]
 [ 8.32083569e-01 -3.84389710e-01  3.98966136e-01  1.00000000e+03]
 [-6.61250828e-01  2.97696231e-01  6.88643194e-01  1.00000000e+03]
 [-6.08547961e-01  7.68096746e-01  2.01305648e-01  1.00000000e+03]
 [-9.05025026e-01  2.02983948e-01  3.74357968e-01  1.00000000e+03]
 [-6.66747839e-01 -7.33181579e-01  1.36946547e-01  1.00000000e

<ipython-input-7-3960a0cab285>:18: RuntimeWarning: invalid value encountered in true_divide
  bvecs_tr_norm[i, :] = bvecs_tr[i, :] / norm[i]


In [10]:
from eddymotion import dmri
data = dmri.load(
    str(dwi_file),
    gradients_file=str(rasb_file),
    b0_file=str(mean_B0_file),
    brainmask_file=str(brain_mask_file)
    )

In [11]:
from eddymotion.estimator import EddyMotionEstimator
EddyMotionEstimator.fit(data, model="tensor", n_threads=6)

  0%|          | 0/30 [00:00<?, ?dwi/s]

Pass 1/1 | Processing b-index <18> in </var/folders/vy/0bw_1jvj54n8lvcgvdrtqb0c0000gn/T/tmpjl1uey6o>


/Applications/miniconda3/lib/python3.8/site-packages/dipy/reconst/dti.py:1955: RuntimeWarning: divide by zero encountered in log
  D[..., 6] = -np.log(b0)
  0%|          | 0/30 [05:34<?, ?dwi/s]


KeyError: 'AffineTransform_float_3_3'